<a href="https://colab.research.google.com/github/jonatas07rocha/passagem-biblica/blob/main/passagem_biblica_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
import os
from google.colab import userdata
import google.generativeai as genai

# Configurações da API do Gemini
gemini_api_key = userdata.get('GOOGLE_GEMINI_API_KEY')
genai.configure(api_key=gemini_api_key)

bible_books = {
    "genesis": "genesis", "gênesis": "genesis",
    # ... (restante do dicionário bible_books)
}

def get_bible_chapter(book, chapter):
  """
  Busca um capítulo da Bíblia A Mensagem.
  """
  normalized_book = bible_books.get(book.lower(), None)
  if not normalized_book:
    print(f"Livro não encontrado: {book}")
    return None

  url = f"https://biblia-a-mensagem.com/{normalized_book}/{chapter}"
  response = requests.get(url)
  if response.status_code != 200:
    print(f"Erro ao acessar a página: {response.status_code}")
    return None

  soup = BeautifulSoup(response.content, 'html.parser')

  # Tratar a página inicial do livro
  if chapter == "1":
    chapter_title = soup.find('h2').text.strip()
    verses = soup.find_all('p')[1:]  # Ignorar o primeiro parágrafo
  else:
    chapter_title = soup.find('h3').text.strip()
    verses = soup.find_all('p')

  formatted_text = f"## {book.title()}\n\n### {chapter_title}\n\n"
  for verse in verses:
    formatted_text += verse.text.strip() + "\n\n"

  return formatted_text

def summarize_chapter(chapter_text):
  """
  Resume o capítulo usando o Google Gemini.
  """

  response = genai.generate_text(
      model="models/gemini-pro",  # Correção aqui!
      prompt=f"Por favor, resuma o seguinte capítulo da Bíblia:\n\n{chapter_text}",
      temperature=0.7,
      max_output_tokens=200,
  )

  return response.text

if __name__ == "__main__":
  book = input("Digite o livro da Bíblia: ")
  chapter = input("Digite o capítulo: ")

  chapter_text = get_bible_chapter(book, chapter)
  if chapter_text:
    print(chapter_text)

    summarize = input("Deseja um resumo deste capítulo? (s/n): ")
    if summarize.lower() == 's':
      summary = summarize_chapter(chapter_text)
      if summary:
        print("\n## Resumo:\n\n", summary)